# Attention is All You Need

Coding a transformer from scratch


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

### Data preprocessing pipeline


In [2]:
# Get Shakespeares work from Andrej Karpathy's website

url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
filepath = tf.keras.utils.get_file('shakespeare.txt', url)

with open(filepath) as f:
    shakespeare_text = f.read()

In [3]:
# Print the first few characters
print(shakespeare_text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [12]:
unique_chars = sorted(set(shakespeare_text))
char_to_int = {char: idx for idx, char in enumerate(unique_chars)}

# How many number of distinct characters has the vocabulary:
tokens_len = len(unique_chars)
print(f'Number of tokens in vocabulary: {tokens_len}')

# How many characters has the dataset:
text_length = len(shakespeare_text)
print(f'Total length of text dataset: {text_length}')

Number of tokens in vocabulary: 65
Total length of text dataset: 1115394


### Embedding Layer


In [ ]:
embedding_dim = 10

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(tokens_len, embedding_dim))

input_array = np.random.randint(tokens_len, size=(1, 1))
model.compile('rmsprop', 'sparse_categorical_crossentropy')

output_array = model.predict(input_array)
print(output_array.shape)

model.summary()
# (1, 1, 10)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
(1, 1, 10)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ (1, 1, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 650 (2.54 KB)

 Trainable params: 650 (2.54 KB)

 Non-trainable params: 0 (0.00 B)

### Positional Encoding


In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, max_pos_enc, embedding_dim, **kwargs):
        super(PositionalEncoding, self).__init__(**kwargs)
        self.max_len = max_pos_enc # maximum sequence length that the model can handle
        self.embedding_dim = embedding_dim
        
        # Create the positional encodings
        position = np.arange(max_pos_enc)[:, np.newaxis]
        div_term = np.exp(np.arange(0, embedding_dim, 2) * -(np.log(10000.0) / embedding_dim))
        pe = np.zeros((max_pos_enc, embedding_dim))
        pe[:, 0::2] = np.sin(position * div_term)
        pe[:, 1::2] = np.cos(position * div_term)
        
        # Add batch dimension e.g. (max_pos_enc,embedding_dim) -> (1,max_pos_enc,embedding_dim)
        self.pe = tf.constant(pe[np.newaxis, :, :], dtype=tf.float32)
    
    def call(self, inputs):
        seq_len = tf.shape(inputs)[1]  # Get sequence length from input
        return inputs + self.pe[:, :seq_len, :]

In [ ]:
embedding_dim = 10  # Embedding dimension
max_len = 50  # Maximum sequence length

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokens_len, embedding_dim),
    PositionalEncoding(max_len, embedding_dim)
])

input_array = np.random.randint(tokens_len, size=(1, 10))  # Example input
output_array = model(input_array)

print(output_array.shape)  # Should be (1, 10, embedding_dim)

(1, 10, 10)


In [27]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ (1, 10, 10)            │           650 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ positional_encoding_3           │ (1, 10, 10)            │             0 │
│ (PositionalEncoding)            │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 650 (2.54 KB)

 Trainable params: 650 (2.54 KB)

 Non-trainable params: 0 (0.00 B)

## Multi-Head Attention


### Scaled Dot-Product Attention


In [8]:
class ScaledDotProductAttention(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(ScaledDotProductAttention, self).__init__(**kwargs)
        
    def call(self, inputs, mask=None):
        q, k, v = inputs
        
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        
        # dot product attention
        matmul_qk = tf.matmul(q, k, transpose_b=True) # (bs, q_len, k_len)
        
        # scale dot product
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
        
        # apply mask when necessary
        if mask is not None:
            # adding very large negative values 
            # so they go to zero after softmax
            scaled_attention_logits += (mask * -1e9) 
        
        # apply softmax to attention weights (scores)
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
        
        # multiply by V (values)
        out = tf.matmul(attention_weights, v)
        
        return out, attention_weights
        

### Multi Head Attention Head


In [9]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, num_heads, embedding_dim, **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        
        self.num_heads = num_heads
        self.embedding_dim = embedding_dim
        assert embedding_dim % num_heads == 0, "embedding_dim must be divisible by num_heads"
        self.depth = embedding_dim // num_heads # depth per head
        
        # linear projection layers
        self.wq = tf.keras.layers.Dense(embedding_dim)
        self.wk = tf.keras.layers.Dense(embedding_dim)
        self.wv = tf.keras.layers.Dense(embedding_dim)
        
        # output projection
        self.dense = tf.keras.layers.Dense(embedding_dim)
        
        self.attention = ScaledDotProductAttention()
    
    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
        
    def call(self, inputs, mask=None):
        q, k, v = inputs
        
        batch_size = tf.shape(q)[0]
        
        # linear projections
        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)
        
        # reshaping q, k, v
        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)
        
        scaled_attention, attention_weights = self.attention([q, k, v])
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embedding_dim))
        
        out = self.dense(concat_attention)
        
        return out, attention_weights

## Position-wise Feed-Forward Network

$$\text{FFN(x)} = \text{max}(0,~ xW_1 + b_1)W_2 + b_2$$


In [10]:
class PositionwiseFeedForward(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, hidden_dim, **kwargs):
        super(PositionwiseFeedForward, self).__init__(**kwargs)
        
        # hidden_dim (dff) - feed forward network hidden 
        # layer dimension a.k.a inner layer dimensionality
        
        self.dense1 = tf.keras.layers.Dense(hidden_dim, activation="relu")
        self.dense2 = tf.keras.layers.Dense(embedding_dim)
        
    def call(self, inputs):
        
        x = self.dense1(inputs)
        
        return self.dense2(x)

## Encoder Layer


In [11]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, hidden_dim, num_heads, dropout_rate = 0.1, **kwargs):
        super(EncoderLayer, self).__init__(**kwargs)
        
        self.mha = MultiHeadAttention(num_heads, embedding_dim)
        self.ffn = PositionwiseFeedForward(embedding_dim, hidden_dim)
        
        self.layer_norm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layer_norm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        
        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
        self.dropout2 = tf.keras.layers.Dropout(dropout_rate)
        
    def call(self, inputs, training=False, mask=None):
        
        # multi-head attention
        attention_output, _ = self.mha([inputs, inputs, inputs], mask)
        attention_output = self.dropout1(attention_output, training=training)
        out1 = self.layer_norm1(inputs + attention_output)
        
        # feed-forward network
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layer_norm2(out1 + ffn_output)
        
        return out2
        

# Encoder

The Encoder stacks multiple encoder layer to create the full encoder. It includes the Embedding layer and Positional Encoding layer as well.


In [ ]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, embedding_dim, hidden_dim, num_heads, 
                 tokens_len, max_pos_enc, dropout_rate=0.1, **kwargs):
        super(Encoder, self).__init__(**kwargs)
            
        self.embedding_dim = embedding_dim
        self.num_layers = num_layers
        
        self.embedding = tf.keras.layers.Embedding(tokens_len, embedding_dim)
        self.pos_encoding = PositionalEncoding(max_pos_enc, embedding_dim)
        
        self.encoding_layers = [
            EncoderLayer(embedding_dim, hidden_dim, num_heads, dropout_rate=dropout_rate)
            for _ in range(num_layers)
        ]
        
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        
    def call(self, x, training=False, mask=None):
        
        seq_len = tf.shape(x)[1]
        
        x = self.embedding(x)
        x = self.pos_encoding(x)
        
        x = self.dropout(x, training=training)
        
        for i in range(self.num_layers):
            x = self.encoding_layers[i](x, training, mask)
            
        return x